In [359]:
train = read.csv("C:\\Users\\shrey\\OneDrive\\Desktop\\BUSINESS ANALYTICS\\StudentTestFiles_tobeScored.csv", na.strings = c("","NA"))

In [360]:
library(caret)
library(ROSE)
library(dummies)

In [361]:
head(train,1)

id_student,code_module,code_presentation,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,...,date_registration,date_unregistration,module_presentation_length,id_assessment,date_submitted,is_banked,score,assessment_type,date_assessment,weight
104215,DDD,2014J,M,Scotland,A Level or Equivalent,0-10%,0-35,0,270,...,-103,-43,262,NA,NA,NA,NA,NA,NA,NA


In [362]:
names(train)

[1] "id_student"                 "code_module"               
 [3] "code_presentation"          "gender"                    
 [5] "region"                     "highest_education"         
 [7] "imd_band"                   "age_band"                  
 [9] "num_of_prev_attempts"       "studied_credits"           
[11] "disability"                 "id_site"                   
[13] "date_vle"                   "sum_click"                 
[15] "activity_type"              "week_from"                 
[17] "week_to"                    "date_registration"         
[19] "date_unregistration"        "module_presentation_length"
[21] "id_assessment"              "date_submitted"            
[23] "is_banked"                  "score"                     
[25] "assessment_type"            "date_assessment"           
[27] "weight"

In [363]:
train= train[!is.na(train$activity_type),]
train= train[!is.na(train$assessment_type),]

In [364]:
#creating dummy for activity_type 
dum = dummy(train$activity_type,sep="_")

Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"

In [365]:
train = cbind(train,dum)

In [366]:
#Creating dummy for assessment_type
dum = dummy(train$assessment_type,sep = "_")

Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"

In [367]:
train = cbind(train,dum)

In [368]:
head(train)

,id_student,code_module,code_presentation,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,...,NA_ouwiki,NA_page,NA_questionnaire,NA_quiz,NA_resource,NA_subpage,NA_url,NA_CMA,NA_Exam,NA_TMA
2,106588,BBB,2013J,F,Scotland,A Level or Equivalent,60-70%,35-55,0,60,...,0,0,0,0,0,0,0,1,0,0
3,108207,FFF,2014B,M,South Region,No Formal quals,50-60%,0-35,0,120,...,0,0,0,0,0,0,0,1,0,0
4,108497,EEE,2014J,M,South West Region,Lower Than A Level,0-10%,0-35,0,60,...,0,0,0,0,0,0,0,0,0,1
5,110524,DDD,2014J,M,Ireland,Lower Than A Level,50-60%,35-55,0,60,...,0,0,0,0,1,0,0,0,1,0
6,111926,DDD,2014J,M,East Anglian Region,Lower Than A Level,30-40%,0-35,2,60,...,0,0,0,0,0,0,0,0,0,1
7,113295,AAA,2013J,M,Yorkshire Region,Lower Than A Level,60-70%,35-55,0,60,...,0,0,0,0,0,0,0,0,0,1


In [369]:
#replacing unregistered NA with "completed"
train[is.na(train$date_unregistration),"date_unregistration"] = "Completed"
#removing unwanted columns
train= train[,-which(names(train) %in% c("week_from","week_to","date_vle","date_submitted","is_banked","date_assessment"))]
names(train)

[1] "id_student"                 "code_module"               
 [3] "code_presentation"          "gender"                    
 [5] "region"                     "highest_education"         
 [7] "imd_band"                   "age_band"                  
 [9] "num_of_prev_attempts"       "studied_credits"           
[11] "disability"                 "id_site"                   
[13] "sum_click"                  "activity_type"             
[15] "date_registration"          "date_unregistration"       
[17] "module_presentation_length" "id_assessment"             
[19] "score"                      "assessment_type"           
[21] "weight"                     "NA_externalquiz"           
[23] "NA_forumng"                 "NA_glossary"               
[25] "NA_homepage"                "NA_oucollaborate"          
[27] "NA_oucontent"               "NA_ouwiki"                 
[29] "NA_page"                    "NA_questionnaire"          
[31] "NA_quiz"                    "NA_resource"               
[33] "NA_subpage"                 "NA_url"                    
[35] "NA_CMA"                     "NA_Exam"                   
[37] "NA_TMA"

In [370]:
#replacing NA score with mean of other NON NA scores
train[is.na(train$score),"score"] = mean(train[!is.na(train$score),"score"])

In [371]:
# Replacing mode of north region imd_band with missing imd_band
summary(train[train$region=="North Region"& !is.na(train$imd_band),"imd_band"])
train[is.na(train$imd_band),"imd_band"] ="10-20 %"

0-10% 10-20 %  20-30%  30-40%  40-50%  50-60%  60-70%  70-80%  80-90% 90-100% 
      2       4       0       2       3       1       2       0       1       0

In [372]:
#no NA values
sapply(train, function(x) sum(is.na(x)))

id_student                code_module 
                         0                          0 
         code_presentation                     gender 
                         0                          0 
                    region          highest_education 
                         0                          0 
                  imd_band                   age_band 
                         0                          0 
      num_of_prev_attempts            studied_credits 
                         0                          0 
                disability                    id_site 
                         0                          0 
                 sum_click              activity_type 
                         0                          0 
         date_registration        date_unregistration 
                         0                          0 
module_presentation_length              id_assessment 
                         0                          0 
                     score            assessment_type 
                         0                          0 
                    weight            NA_externalquiz 
                         0                          0 
                NA_forumng                NA_glossary 
                         0                          0 
               NA_homepage           NA_oucollaborate 
                         0                          0 
              NA_oucontent                  NA_ouwiki 
                         0                          0 
                   NA_page           NA_questionnaire 
                         0                          0 
                   NA_quiz                NA_resource 
                         0                          0 
                NA_subpage                     NA_url 
                         0                          0 
                    NA_CMA                    NA_Exam 
                         0                          0 
                    NA_TMA 
                         0

In [373]:
#train$TMA_score = train$score

In [374]:
#train= train[,-which(names(train) %in% c("score"))]

In [375]:
#putting scores in the assessment
train$TMA_score = train$score* train$NA_TMA
train$Exam_score= train$score* train$NA_Exam
train$CMA_Score= train$score* train$NA_CMA

In [376]:
From_model = read.csv("C:\\Users\\shrey\\OneDrive\\Desktop\\BUSINESS ANALYTICS\\Datasets\\Project2\\final_data2.csv", na.strings = c("","NA"))

In [377]:
train= train[,-which(names(train) %in% c("NA_CMA","NA_Exam","NA_TMA","id_site","activity_type","id_assessment","score","assessment_type"))]

In [378]:

From_model= From_model[,-which(names(From_model) %in% c("key","X","CMA_count","Exam_count","TMA_count","NA_sharedsubpage",
                                                        "NA_dataplus","NA_dualpane","NA_folder","NA_htmlactivity","NA_ouelluminate",
                                                        "NA_repeatactivity"))]

In [379]:
length(names(From_model))
length(names(train))

[1] 33

[1] 32

In [380]:
names(From_model)
names(train)

[1] "code_module.x.x"            "code_presentation.x.x"     
 [3] "id_student.x"               "date_registration"         
 [5] "date_unregistration"        "gender"                    
 [7] "region"                     "highest_education"         
 [9] "imd_band"                   "age_band"                  
[11] "num_of_prev_attempts"       "studied_credits"           
[13] "disability"                 "final_result"              
[15] "sum_click"                  "NA_externalquiz"           
[17] "NA_forumng"                 "NA_glossary"               
[19] "NA_homepage"                "NA_oucollaborate"          
[21] "NA_oucontent"               "NA_ouwiki"                 
[23] "NA_page"                    "NA_questionnaire"          
[25] "NA_quiz"                    "NA_resource"               
[27] "NA_subpage"                 "NA_url"                    
[29] "weight"                     "TMA_score"                 
[31] "Exam_score"                 "CMA_Score"                 
[33] "module_presentation_length"

[1] "id_student"                 "code_module"               
 [3] "code_presentation"          "gender"                    
 [5] "region"                     "highest_education"         
 [7] "imd_band"                   "age_band"                  
 [9] "num_of_prev_attempts"       "studied_credits"           
[11] "disability"                 "sum_click"                 
[13] "date_registration"          "date_unregistration"       
[15] "module_presentation_length" "weight"                    
[17] "NA_externalquiz"            "NA_forumng"                
[19] "NA_glossary"                "NA_homepage"               
[21] "NA_oucollaborate"           "NA_oucontent"              
[23] "NA_ouwiki"                  "NA_page"                   
[25] "NA_questionnaire"           "NA_quiz"                   
[27] "NA_resource"                "NA_subpage"                
[29] "NA_url"                     "TMA_score"                 
[31] "Exam_score"                 "CMA_Score"

In [381]:
colnames(From_model)[1] = 'code_module'
colnames(From_model)[2] = 'code_presentation'
colnames(From_model)[3] = 'id_student'

In [382]:
names(From_model)
names(train)

[1] "code_module"                "code_presentation"         
 [3] "id_student"                 "date_registration"         
 [5] "date_unregistration"        "gender"                    
 [7] "region"                     "highest_education"         
 [9] "imd_band"                   "age_band"                  
[11] "num_of_prev_attempts"       "studied_credits"           
[13] "disability"                 "final_result"              
[15] "sum_click"                  "NA_externalquiz"           
[17] "NA_forumng"                 "NA_glossary"               
[19] "NA_homepage"                "NA_oucollaborate"          
[21] "NA_oucontent"               "NA_ouwiki"                 
[23] "NA_page"                    "NA_questionnaire"          
[25] "NA_quiz"                    "NA_resource"               
[27] "NA_subpage"                 "NA_url"                    
[29] "weight"                     "TMA_score"                 
[31] "Exam_score"                 "CMA_Score"                 
[33] "module_presentation_length"

[1] "id_student"                 "code_module"               
 [3] "code_presentation"          "gender"                    
 [5] "region"                     "highest_education"         
 [7] "imd_band"                   "age_band"                  
 [9] "num_of_prev_attempts"       "studied_credits"           
[11] "disability"                 "sum_click"                 
[13] "date_registration"          "date_unregistration"       
[15] "module_presentation_length" "weight"                    
[17] "NA_externalquiz"            "NA_forumng"                
[19] "NA_glossary"                "NA_homepage"               
[21] "NA_oucollaborate"           "NA_oucontent"              
[23] "NA_ouwiki"                  "NA_page"                   
[25] "NA_questionnaire"           "NA_quiz"                   
[27] "NA_resource"                "NA_subpage"                
[29] "NA_url"                     "TMA_score"                 
[31] "Exam_score"                 "CMA_Score"

In [383]:
#converting distinction to pass and witdrawn as fail
From_model$final_result = as.factor(From_model$final_result )
From_model[From_model$final_result=="Distinction","final_result"] = "Pass"
From_model[From_model$final_result=="Withdrawn","final_result"] = "Fail"

In [384]:
unique(From_model$final_result)

[1] Pass Fail
Levels: Distinction Fail Pass Withdrawn

In [331]:
write.csv(From_model,"C:\\Users\\shrey\\OneDrive\\Desktop\\Prof_train_data.csv", row.names = FALSE)

In [332]:
From_model = read.csv("C:\\Users\\shrey\\OneDrive\\Desktop\\Prof_train_data.csv")

In [294]:
#splitting the Model data
## 90% of the sample size
smp_size <- floor(0.90 * nrow(From_model))

## set the seed to make your partition reproducible
set.seed(123)
train_ind <- sample(seq_len(nrow(From_model)), size = smp_size)

Train <- From_model[train_ind, ]
Test <- From_model[-train_ind, ]

row.names(Train) = NULL
row.names(Test)= NULL

In [295]:
# Run algorithms using 10-fold cross validation
control <- trainControl(method="cv", number=10)
metric <- "Accuracy"

In [296]:
# GLM with balanced data
set.seed(7)
fit.GBM1 <- train(final_result~., data=Train, method="gbm", metric=metric, trControl=control)

Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 24: date_unregistration-112 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 30: date_unregistration-118 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 50: date_unregistration-141 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 73: date_unregistration-179 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 80: date_unregistration-194 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 95: date_unregistration-225 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"varia

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        1.2879             nan     0.1000    0.0439
     2        1.2160             nan     0.1000    0.0356
     3        1.1569             nan     0.1000    0.0291
     4        1.1091             nan     0.1000    0.0241
     5        1.0690             nan     0.1000    0.0198
     6        1.0346             nan     0.1000    0.0172
     7        1.0060             nan     0.1000    0.0142
     8        0.9820             nan     0.1000    0.0120
     9        0.9588             nan     0.1000    0.0116
    10        0.9381             nan     0.1000    0.0103
    20        0.8155             nan     0.1000    0.0035
    40        0.7254             nan     0.1000    0.0015
    60        0.6869             nan     0.1000    0.0006
    80        0.6681             nan     0.1000    0.0002
   100        0.6576             nan     0.1000    0.0001
   120        0.6502             nan     0.1000    0.0001
   140        

Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 24: date_unregistration-112 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 30: date_unregistration-118 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 50: date_unregistration-141 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 73: date_unregistration-179 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 80: date_unregistration-194 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 95: date_unregistration-225 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"varia

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        1.2750             nan     0.1000    0.0502
     2        1.1923             nan     0.1000    0.0411
     3        1.1249             nan     0.1000    0.0339
     4        1.0680             nan     0.1000    0.0285
     5        1.0180             nan     0.1000    0.0249
     6        0.9772             nan     0.1000    0.0206
     7        0.9398             nan     0.1000    0.0185
     8        0.9083             nan     0.1000    0.0158
     9        0.8827             nan     0.1000    0.0128
    10        0.8589             nan     0.1000    0.0118
    20        0.7336             nan     0.1000    0.0035
    40        0.6582             nan     0.1000    0.0011
    60        0.6312             nan     0.1000    0.0005
    80        0.6138             nan     0.1000    0.0003
   100        0.6015             nan     0.1000    0.0002
   120        0.5947             nan     0.1000    0.0001
   140        

Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 24: date_unregistration-112 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 30: date_unregistration-118 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 50: date_unregistration-141 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 73: date_unregistration-179 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 80: date_unregistration-194 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 95: date_unregistration-225 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"varia

...

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        1.2755             nan     0.1000    0.0500
     2        1.1943             nan     0.1000    0.0408
     3        1.1263             nan     0.1000    0.0338
     4        1.0695             nan     0.1000    0.0282
     5        1.0209             nan     0.1000    0.0244
     6        0.9789             nan     0.1000    0.0204
     7        0.9434             nan     0.1000    0.0178
     8        0.9133             nan     0.1000    0.0150
     9        0.8872             nan     0.1000    0.0130
    10        0.8638             nan     0.1000    0.0117
    20        0.7396             nan     0.1000    0.0032
    40        0.6651             nan     0.1000    0.0009
    60        0.6372             nan     0.1000    0.0003
    80        0.6235             nan     0.1000    0.0001
   100        0.6126             nan     0.1000    0.0006
   120        0.6047             nan     0.1000    0.0002
   140        

Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 24: date_unregistration-112 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 31: date_unregistration-119 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 73: date_unregistration-179 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 79: date_unregistration-192 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 88: date_unregistration-207 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 95: date_unregistration-225 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"varia

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        1.2693             nan     0.1000    0.0527
     2        1.1837             nan     0.1000    0.0430
     3        1.1112             nan     0.1000    0.0361
     4        1.0508             nan     0.1000    0.0300
     5        1.0010             nan     0.1000    0.0250
     6        0.9571             nan     0.1000    0.0219
     7        0.9174             nan     0.1000    0.0196
     8        0.8855             nan     0.1000    0.0160
     9        0.8576             nan     0.1000    0.0139
    10        0.8330             nan     0.1000    0.0123
    20        0.6994             nan     0.1000    0.0036
    40        0.6349             nan     0.1000    0.0008
    60        0.6118             nan     0.1000    0.0003
    80        0.5992             nan     0.1000   -0.0000
   100        0.5869             nan     0.1000   -0.0000
   120        0.5787             nan     0.1000    0.0000
   140        

Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 24: date_unregistration-112 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 73: date_unregistration-179 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 95: date_unregistration-225 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 336: date_unregistration239 has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 441: imd_bandTRUE has no variation."

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        1.2677             nan     0.1000    0.0535
     2        1.1811             nan     0.1000    0.0434
     3        1.1102             nan     0.1000    0.0356
     4        1.0500             nan     0.1000    0.0301
     5        0.9986             nan     0.1000    0.0255
     6        0.9528             nan     0.1000    0.0227
     7        0.9155             nan     0.1000    0.0186
     8        0.8810             nan     0.1000    0.0171
     9        0.8525             nan     0.1000    0.0142
    10        0.8287             nan     0.1000    0.0118
    20        0.6980             nan     0.1000    0.0038
    40        0.6304             nan     0.1000    0.0008
    60        0.6086             nan     0.1000    0.0001
    80        0.5939             nan     0.1000    0.0003
   100        0.5843             nan     0.1000    0.0003
   120        0.5778             nan     0.1000    0.0000
   140        

In [297]:
saveRDS(fit.GBM1, "C:\\Users\\shrey\\OneDrive\\Desktop\\BUSINESS ANALYTICS\\Datasets\\models\\project2\\testcasemodel.rds")

In [301]:
library(pROC)

Type 'citation("pROC")' for a citation.

Attaching package: 'pROC'

The following objects are masked from 'package:stats':

    cov, smooth, var



In [391]:
train$imd_band = FALSE

In [397]:
df_test_prof = data.frame(train$id_student)

In [398]:
df_test_prof$predicted= predict(fit.GBM1, train)

In [399]:
head(df_test_prof)

train.id_student,predicted
106588,Fail
108207,Fail
108497,Fail
110524,Fail
111926,Fail
113295,Fail


In [401]:
write.csv(df_test_prof,"C:\\Users\\shrey\\OneDrive\\Desktop\\Pro_predicted.csv", row.names = FALSE)